- 데이터 파일을 읽어 질문과 ID를 추출
- InferSent 모델 및 Google USE 임베딩 함수 준비
- 분류기(MLP) 모델 로드
- 질문을 배치 단위로 임베딩 생성 → 분류기 예측 → 질문 유형(q_type) 리스트 생성
- 결과를 원본 데이터에 반영하여 새 파일로 저장

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="3"
from __future__ import absolute_import, division
from tensorflow.python.framework.ops import enable_eager_execution
enable_eager_execution()
import io
import os
import sys
import logging
import json
import torch
import nltk
import numpy as np
import re
from infersent import InferSent
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity(0)
import tensorflow_hub as hub
import warnings
warnings.filterwarnings('ignore')
# nltk.download('punkt_tab')

# Set PATHs
PATH_SENTEVAL = '../../utils/SentEval'  # specify SentEval root if not installed
PATH_TO_DATA = ''  # not necessary for inference
MODEL_VERSION = 1
# 300차원의 사전 단어 임베딩 벡터
PATH_TO_W2V = '../../data/glove/glove.840B.300d.txt' if MODEL_VERSION == 1 else '../SentEval/fasttext/crawl-300d-2M.vec'
# 사전 학습된 임베딩 모델 Infersent1
MODEL_PATH = "../../model/encoder/infersent%s.pkl" % MODEL_VERSION

## glove.txt: 각 단어를 300차원의 벡터로 표현하는 사전 임베딩 파일
# ->infersent모델이 입력 문장 임베딩할 때, 각 단어 벡터로 변환에 사용
## infersent1.pkl: 사전 학습된 문장 임베딩 모델의 가중치 파일
# ->여러 단어 임베딩(GloVe)받아 문장 전체의 의미를 하나의 벡터로 변환
# -> GloVe:단어를 벡터로 / InferSent:문장을 벡터로

sys.path.insert(0, PATH_SENTEVAL)
import senteval
from senteval.tools.classifier import MLP

# 경로(폴더 위치), 현재 작업 경로
print('current path: ', os.getcwd())
# 모듈 찾을 때 참조하는 경로 리스트, 상대 경로 해석(sys.path.insert로 수정)
# sys.path

2025-09-25 12:45:10.438780: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


current path:  /home/gayeon39/gayeon/[DA]/code/clf_question


In [2]:
def clean_sentence(text: str) -> str:
    # 소유격, 축약형 's 분리
    text = re.sub(r"(\w+)'s", r"\1 's", text)
    # 축약형 단순 변환s
    text = text.replace("n't", " not")
    text = text.replace("'re", " are")
    text = text.replace("'ve", " have")
    text = text.replace("'ll", " will")
    text = text.replace("'d", " would")
    text = text.replace("'m", " am")
    text = text.replace("What's", "What is") 
    # 하이픈 단어 분리
    text = re.sub(r"(\w+)-(\w+)", r"\1 \2", text)
    # 불필요한 특수문자 제거
    text = re.sub(r"[^A-Za-z0-9\s\?]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def preprocess_sentences(sent_list):
    return [clean_sentence(s) for s in sent_list]

def loadFile(fpath):
    """
    MRQA 스타일 데이터 파일을 읽어 질문과 질문 ID를 추출
    파일 한 줄씩 읽어 JSON으로 파싱->각 질문 토큰화하여 저장
    """
    qa_data = []
    qa_ids = []
    # tgt2idx = {'ABBR': 0, 'DESC': 1, 'ENTY': 2,
    #             'HUM': 3, 'LOC': 4, 'NUM': 5}
    with io.open(fpath, 'r', encoding='utf-8') as f:
        for example in f:
            if "header" in json.loads(example):
                continue
            paragraph = json.loads(example)
            for qa in paragraph['qas']:
                qa_data.append(qa['question'].split())
                qa_ids.append(qa['qid'])
    return qa_data, qa_ids

def loadSQuAD(fpath):
    """
    SQuAD 스타일 데이터 파일을 읽어 질문과 질문 ID를 추출
    파일 한 줄씩 읽어 JSON으로 파싱->각 질문 토큰화하여 저장
    """
    qa_data = []
    qa_ids = []
    # tgt2idx = {'ABBR': 0, 'DESC': 1, 'ENTY': 2,
    #             'HUM': 3, 'LOC': 4, 'NUM': 5}
    with io.open(fpath, 'r', encoding='utf-8') as f:
        input_data = json.load(f)["data"]
        input_data = input_data ################ 개수 줄이기 ################ 
    for entry in input_data:
        for paragraph in entry["paragraphs"]:
            for qa in paragraph["qas"]:
                qa_data.append(qa['question'].split())
                qa_ids.append(qa['id'])
    return qa_data, qa_ids

def prepare(params, samples):
    # InferSent 모델에 사용할 단어 사전을 구축하는 함수
    # samples: 문장(질문) 리스트
    # 축약형·소유격·하이픈 결합 단어 전처리 
    sentences = [' '.join(sent) if sent != [] else '.' for sent in samples]
    sentences = preprocess_sentences(sentences)
    params['infersent'].build_vocab(sentences, tokenize=False) # 모델 불러옴

def batcher(params, batch):
    # 입력된 질문 배치를 임베딩 벡터로 변환하는 함수
    # 1. 각 질문(토큰 리스트)을 문자열로 합침
    sentences = [' '.join(sent) if sent != [] else '.' for sent in batch]
    # 2. 축약형·소유격·하이픈 결합 단어 전처리
    sentences = preprocess_sentences(sentences)
    # 3. InferSent 모델로 임베딩 생성
    embeddings1 = params['infersent'].encode(sentences, bsize=params['classifier']['batch_size'], tokenize=False)
    # 4. Google Universal Sentence Encoder로 임베딩 생성
    embeddings2 = params['google_use'](sentences)
    embeddings2 = embeddings2.numpy()
    # 5. 두 임베딩을 합쳐서 반환 (문장 의미를 더 풍부하게 표현)
    return np.concatenate((embeddings1, embeddings2), axis=-1)

def make_embed_fn(module):
    embed = hub.load(module)
    f = embed.signatures["default"]
    return lambda x: f(tf.constant(x))["default"]

def getEmbeddings(qa_data, params):
    # 전체 질문 리스트를 배치 단위로 나눠 임베딩을 생성하는 함수
    out_embeds = []
    # 배치 크기만큼 반복하며 임베딩 생성
    for ii in range(0, len(qa_data), params['classifier']['batch_size']):
        batch = qa_data[ii:ii + params['classifier']['batch_size']]
        # batcher 함수로 임베딩 생성
        embeddings = batcher(params, batch)
        out_embeds.append(embeddings)
    # 모든 배치 임베딩을 하나로 합쳐 반환(행렬 세로 방향으로 이어붙임)
    # 입력: [batch_size개의 문장] -> 출력: (batch_size, emb_dim)
    return np.vstack(out_embeds) # (N, 4096+512)

def updateFile(fpath, q_type, q_type_prob,q_ids):
    # MRQA 스타일 데이터 파일에 질문 유형(q_type)을 추가하는 함수
    paragraphs = []
    # 원본 파일을 한 줄씩 읽어서 JSON 객체로 파싱
    with io.open(fpath, 'r', encoding='utf-8') as f:
        for example in f:
            # header가 포함된 줄은 건너뜀
            if "header" in json.loads(example):
                continue
            paragraph = json.loads(example)
            paragraphs.append(paragraph)
    total_idx = 0
    # 각 paragraph의 qas 리스트를 순회하며 질문 ID(qid)와 분류 결과(q_type)를 매칭
    for paragraph in paragraphs:
        for qa in paragraph['qas']:
            # 예측된 질문 유형을 해당 질문에 추가
            if qa['qid'] == q_ids[total_idx]:
                qa['q_type'] = q_type[total_idx]
                qa['q_type_prob'] = q_type_prob[total_idx]
                total_idx += 1
            else:
                # 질문 ID가 맞지 않으면 경고 출력
                print('Can not match qid:', q_ids[total_idx])
    # 새로운 파일로 결과 저장 (원본 파일명에서 .jsonl을 _classified.jsonl로 변경)
    with open(fpath[:-6]+'_classified_09151215.jsonl', 'w') as f:
        for sample in paragraphs:
            f.write(json.dumps(sample)+'\n')
    f.close()

def updateSQuAD(fpath, q_type, q_type_prob, q_ids):
    # SQuAD 스타일 데이터 파일에 질문 유형(q_type)과 질문 유형 확률(q_type_prob)을 추가하는 함수
    with io.open(fpath, 'r', encoding='utf-8') as f:
        input_data = json.load(f)["data"]
        input_data = input_data ################ 개수 줄이기 ################
    total_idx = 0
    # SQuAD 구조에 맞게 각 질문에 대해 분류 결과를 추가
    for entry in input_data:
        for paragraph in entry["paragraphs"]:
            for qa in paragraph['qas']:
                # 예측된 질문 유형을 해당 질문에 추가
                if qa['id'] == q_ids[total_idx]:
                    qa['q_type'] = q_type[total_idx]
                    qa['q_type_prob'] = q_type_prob[total_idx]
                    total_idx += 1
                else:
                    # 질문 ID가 맞지 않으면 경고 출력
                    print('Can not match qid:', q_ids[total_idx])
    
    # 새로운 파일로 결과 저장 (원본 파일명에서 .json을 _classified.json로 변경)
    with open(fpath[:-5]+'_classified_qtype_prob.json', 'w') as f:
        f.write(json.dumps({"data": input_data})+'\n')
    f.close()

In [3]:
# Set
encoder = make_embed_fn("https://tfhub.dev/google/universal-sentence-encoder-large/3")
params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 5}
params_senteval['classifier'] = {'nhid': 512, 'optim': 'rmsprop', 'batch_size': 16,
                                 'tenacity': 5, 'epoch_size': 4}
params_senteval['google_use'] = encoder
params_model = {'bsize': 16, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': MODEL_VERSION}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))
model.set_w2v_path(PATH_TO_W2V)
params_senteval['infersent'] = model.cuda().eval()

# Set Parameter
file_path = '../../data/squad/train-v1.1.json'
all_qs, all_q_ids = loadSQuAD(file_path)
q_type = []
q_type_prob = []
clf = MLP(params_senteval['classifier'], inputdim=4096+512, nclasses=6, batch_size=16)
clf.model.load_state_dict(torch.load('../../model/clf_trec/qc4qa_model.pth'))  
clf.model.eval()

I0000 00:00:1758771914.315611 2712616 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22264 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:4c:00.0, compute capability: 8.9


Sequential(
  (0): Linear(in_features=4608, out_features=512, bias=True)
  (1): Dropout(p=0.0, inplace=False)
  (2): Sigmoid()
  (3): Linear(in_features=512, out_features=6, bias=True)
)

In [4]:
with torch.no_grad(): 
    # 질문을 배치 단위로 임베딩->MLP예측->결과 누적
    for i in range(0, len(all_qs), 1000): 
        qs = all_qs[i:1000+i] # 현재 배치에 해당하는 질문 1000개 선택
        prepare(params_senteval, qs) # InferSent모델에 맞게 단어 사전 구축
        # 선택된 질문을 InferSent와 Google USE로 임베딩 벡터로 변환
        embeds = getEmbeddings(qs, params_senteval) 
        # 임베딩된 질문을 MLP 분류기에 입력하여 질문 유형 예측
        out = clf.predict(embeds)
        out_proba = clf.predict_proba(embeds)
        out_proba = np.round(out_proba, 3)
        # 예측 결과를 리스트로 변환하여 전체 결과(q_type)에 추가
        q_type += np.array(out).squeeze().astype(int).tolist()
        q_type_prob += out_proba.tolist()
        
# SQuAD sytle update
updateSQuAD(file_path, q_type, q_type_prob, all_q_ids)

Found 1707(/2196) words with w2v vectors
Vocab size : 1707
Found 1978(/2527) words with w2v vectors
Vocab size : 1978
Found 1917(/2453) words with w2v vectors
Vocab size : 1917
Found 1716(/2260) words with w2v vectors
Vocab size : 1716
Found 1951(/2434) words with w2v vectors
Vocab size : 1951
Found 2038(/2617) words with w2v vectors
Vocab size : 2038
Found 1976(/2469) words with w2v vectors
Vocab size : 1976
Found 1558(/1940) words with w2v vectors
Vocab size : 1558
Found 1942(/2459) words with w2v vectors
Vocab size : 1942
Found 2288(/2802) words with w2v vectors
Vocab size : 2288
Found 2075(/2710) words with w2v vectors
Vocab size : 2075
Found 2173(/2739) words with w2v vectors
Vocab size : 2173
Found 2110(/2645) words with w2v vectors
Vocab size : 2110
Found 1786(/2378) words with w2v vectors
Vocab size : 1786
Found 1969(/2470) words with w2v vectors
Vocab size : 1969
Found 1710(/2232) words with w2v vectors
Vocab size : 1710
Found 1980(/2462) words with w2v vectors
Vocab size : 19

In [6]:
file_path = '../../data/squad/train-v1.1_classified_qtype_prob.json'
with io.open(file_path, 'r', encoding='utf-8') as f:
    tmp_data = json.load(f)['data']
tmp_data[0]['paragraphs'][0]['qas'][0]

{'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}],
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'id': '5733be284776f41900661182',
 'q_type': 3,
 'q_type_prob': [0.0, 0.0, 0.016, 0.983, 0.001, 0.0]}